論文<br>
https://arxiv.org/abs/2111.03600<br>
<br>
GitHub<br>
https://github.com/facebookresearch/demucs<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Demucs_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

In [ ]:
!nvidia-smi

## GithubからCode clone

In [ ]:
%cd /content

## ライブラリのインストール

In [ ]:
!pip install demucs
!pip install yt-dlp moviepy
!pip install imageio==2.4.1

## ライブラリのインポート

In [ ]:
from yt_dlp import YoutubeDL

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import IPython.display as ipd

# テスト動画のセットアップ

In [ ]:
video_url = 'https://www.youtube.com/watch?v=bnzPJhW9XQg' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  74#@param {type:"integer"}
end_sec =  104#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

## ダウンロード

In [ ]:
!mkdir -p /content/videos /content/audios

download_resolution = 360
full_video_path = '/content/videos/full_video.mp4'
input_clip_path = '/content/videos/input_clip.mp4'
input_audio_path = '/content/audios/input_clip.wav'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

## clip

In [ ]:
# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

## 音声ファイル保存

In [ ]:
# 音声抽出
clip = VideoFileClip(input_clip_path)
clip.audio.write_audiofile(input_audio_path, codec='pcm_s16le')

ipd.Audio(input_audio_path, rate=44100)

# 音源分離

In [ ]:
!python3 -m demucs --two-stems=vocals {input_audio_path}

## ボーカル再生

In [ ]:
vocal_wav = '/content/separated/mdx_extra_q/input_clip/vocals.wav'
ipd.Audio(vocal_wav, rate=44100)

## ボーカルのみの動画を作成

In [ ]:
vocal_video = '/content/vocal.mp4'

videoclip = VideoFileClip(input_clip_path)
audioclip = AudioFileClip(vocal_wav)

new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile(vocal_video)

In [ ]:
clip = VideoFileClip(vocal_video)
  
clip = resize(clip, height=420)
clip.ipython_display()